In [3]:
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
import numpy as np

assets = ['AAPL', 'TSLA', 'NVDA', 'K','CRSP','NEM','PLTR']
benchmarks = ['SPY','IWM','DIA']

#Date times
#Date functions found on StackOverflow
end_date = dt.now().strftime('%Y-%m-%d')
startdate_3m = pd.to_datetime(end_date) - pd.DateOffset(months = 3)
startdate_12m = pd.to_datetime(end_date) - pd.DateOffset(months = 12)

#3months, 12months, benchmark data
data_3m = yf.download(tickers = assets, start = startdate_3m, end = end_date)['Adj Close']
data_12m = yf.download(tickers = assets, start = startdate_12m, end = end_date)['Adj Close']
benchmarks_data = yf.download(tickers = benchmarks, start = startdate_12m, end = end_date)['Adj Close']

#Portfolio weight
equal_weight = 1 / len(assets)

#Daily returns for volatiltity
returns_3m = data_3m.pct_change().dropna()
returns_12m = data_12m.pct_change().dropna()
returns_benchmark = benchmarks_data.pct_change().dropna()

#Annualized volatility - formula (Standard Deviation * square root of 252)
Annualized_Volatility = returns_3m.std() * np.sqrt(252) 

#StackOverflow used to find the covariance and variance snippets for the beta
#Defined function to find beta by calculating covariance and variance
def calculate_beta(asset_returns, benchmark_returns):
    cov_matrix = np.cov(asset_returns, benchmark_returns)
    cov_stock_benchmark = cov_matrix[0, 1]
    var_benchmark = np.var(benchmark_returns)
    beta = cov_stock_benchmark / var_benchmark
    return beta

#Calculates the beta against the benchmarks
betas_spy = [calculate_beta(returns_12m[ticker], returns_benchmark['SPY']) for ticker in assets]
betas_iwm = [calculate_beta(returns_12m[ticker], returns_benchmark['IWM']) for ticker in assets]
betas_dia = [calculate_beta(returns_12m[ticker], returns_benchmark['DIA']) for ticker in assets]

#Risk analysis table
analysis = pd.DataFrame({
    'Portfolio Weight' : equal_weight,
    'Annualized Volatility (3 months)' : Annualized_Volatility,
    'Beta against SPY' : betas_spy,
    'Beta against IWM' : betas_iwm,
    'Beta against DIA' : betas_dia
})

print (analysis)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  3 of 3 completed

        Portfolio Weight  Annualized Volatility (3 months)  Beta against SPY  \
Ticker                                                                         
AAPL            0.142857                          0.225913          1.000410   
CRSP            0.142857                          0.485624          2.218461   
K               0.142857                          0.395310          2.715886   
NEM             0.142857                          0.290813         -0.279678   
NVDA            0.142857                          0.653355          2.050866   
PLTR            0.142857                          0.532938          0.793367   
TSLA            0.142857                          0.658173          2.571738   

        Beta against IWM  Beta against DIA  
Ticker                                      
AAPL            0.334830          0.810196  
CRSP            1.027943          1.821779  
K               0.650053          1.286536  
NEM            -0.012661          0.016924  
NVDA     